In [9]:
import urllib3
import json

# 언어 분석 기술(문어)
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"

accessKey = "c0d741d0-1225-41da-b291-b1f6039778ae"
analysisCode = "wsd"    # analysisCode: 언어 분석 기술 중 "Word Sense Disambiguation"를 선택
text = "기계공학, 재료공학, 전기공학, 컴퓨터 공학, 디자인(산업 디자인, 제품 디자인 등), 로봇공학, 제조공학, 컴퓨터 그래픽스, 원격 감시 및 제어, 산업공학, 3D 프린팅 공학, 자동차 공학, 캐드/CAM 공학, 의료공학, 생체재료공학" # 임의로 설정(원래는 chatGPT답변)

requestJson = {  
    "argument": {
        "text": text,
        "analysis_code": analysisCode
    }
}

http = urllib3.PoolManager()
response = http.request(
    "POST",
    openApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  accessKey},
    body=json.dumps(requestJson, indent=4)
)

#print("[responseCode] " + str(response.status))

response_data = json.loads(response.data)
#print("[responBody]")
#print(json.dumps(response_data, indent=4, ensure_ascii=False))

data = response.data

parsed_data = json.loads(data)

# "morp" 키에서 "type"이 "NNG"이면서 "lemma"이 "학과"나 "과"가 아닌 항목들만 추출
filtered_morp = [item['lemma'] for item in parsed_data['return_object']['sentence'][0]['morp'] if item['type'] == 'NNG' and item['lemma'] not in ['학과', '과']]

# 중복 요소 삭제
filtered_morp = list(set(filtered_morp))

print(filtered_morp)

MaxRetryError: HTTPConnectionPool(host='aiopen.etri.re.kr', port=8000): Max retries exceeded with url: /WiseNLU (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000259D38C4B80>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [66]:
import time
import requests
import pandas as pd
from xml.etree import ElementTree

url = 'https://api.odcloud.kr/api/15014632/v1/uddi:6939f45b-1283-4462-b394-820c26e1445d'
params = {
    'page': 1,
    'perPage': 7000,
    'returnType': 'XML',
    'serviceKey': 'GjCdPonj23F6SdOIs8TsI4OEUkZIi3oa6PcTvdoGQZQIDMZTgCYzn4HlzpcPKxxxJXECIt+rqU1GNFtm3fB8IA=='
}

data = []

while True:
    response = requests.get(url, params=params)
    xml_data = response.text
    root = ElementTree.fromstring(xml_data)

    for item in root.findall('data/item'):
        item_data = {}
        for col in item.findall('col'):
            name = col.get('name')
            if name == '학교명' or name == '학부_과(전공)명':
                item_data[name] = col.text
        data.append(item_data)

    if int(root.findtext('totalCount')) <= len(data):
        break

    params['page'] += 1

df = pd.DataFrame(data)

df = df[~df['학교명'].fillna('').str.contains('대학원')]
df = df[['학부_과(전공)명', '학교명']]

print(df)

      학부_과(전공)명     학교명
0         기계공학과   조선대학교
1         기계공학과   한밭대학교
2         기계공학과   단국대학교
11        기계공학부  성균관대학교
12        기계공학부   영남대학교
...         ...     ...
52209       화학과   울산대학교
52210     물리학전공   울산대학교
52211      물리학과   울산대학교
52212    자연과학계열   울산대학교
52213  교양대학(진천)   우석대학교

[33043 rows x 2 columns]


In [68]:
#print(text)
#print(filtered_morp)
#filtered_morp = ['자동', '기계', '감시', '산업', '생체', '의료', '전기', '제어', '디자인', '재료', '제품', '원격', '제조', '프린팅', '컴퓨터', '캐드', '그래픽스', '로봇', '공학', '차']
new_df = pd.DataFrame()

# Remove rows with missing values in '학부_과(전공)명' column
df = df.dropna(subset=['학부_과(전공)명'])

# 'filtered_morp' 리스트의 각 항목을 순회하면서 
# DataFrame 'df'에서 해당 학부 또는 전공명을 가진 row들만 추출하여 출력
for word in filtered_morp:
    filtered_rows = df[df['학부_과(전공)명'].str.contains(word)]
    # str.contains(word) 메서드는 해당 열의 각 항목이 word 변수에 저장된 형태소 단어를 포함하는지를 검사
    new_df = new_df.append(filtered_rows)
    #print(filtered_rows)
    print('<', word, '> 분야와 관련하여', len(filtered_rows), '개의 학과와 대학이 검색되었습니다.')   # 임의로 작성
    
# 중복 제거
new_df = new_df.drop_duplicates(subset=['학부_과(전공)명', '학교명'])

print(new_df)
print("\n", '해당 분야와 관련하여 총', len(new_df), '개의 학과와 대학이 검색되었습니다.')
#new_df.to_csv('C:/Users/황의지/Desktop/hk+/1', index=False, encoding='utf-8-sig')

C:\Users\황의지\AppData\Local\Temp\ipykernel_15068\228023947.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(filtered_rows)
C:\Users\황의지\AppData\Local\Temp\ipykernel_15068\228023947.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(filtered_rows)
C:\Users\황의지\AppData\Local\Temp\ipykernel_15068\228023947.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(filtered_rows)
C:\Users\황의지\AppData\Local\Temp\ipykernel_15068\228023947.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(filtered_rows)
C:\Users\황의지\AppData\Local\Temp\ipykernel_15068\

< 자동 > 분야와 관련하여 601 개의 학과와 대학이 검색되었습니다.
< 기계 > 분야와 관련하여 925 개의 학과와 대학이 검색되었습니다.
< 감시 > 분야와 관련하여 0 개의 학과와 대학이 검색되었습니다.
< 산업 > 분야와 관련하여 985 개의 학과와 대학이 검색되었습니다.
< 생체 > 분야와 관련하여 11 개의 학과와 대학이 검색되었습니다.
< 의료 > 분야와 관련하여 359 개의 학과와 대학이 검색되었습니다.
< 전기 > 분야와 관련하여 662 개의 학과와 대학이 검색되었습니다.
< 제어 > 분야와 관련하여 147 개의 학과와 대학이 검색되었습니다.
< 디자인 > 분야와 관련하여 1671 개의 학과와 대학이 검색되었습니다.
< 재료 > 분야와 관련하여 87 개의 학과와 대학이 검색되었습니다.
< 제품 > 분야와 관련하여 27 개의 학과와 대학이 검색되었습니다.
< 원격 > 분야와 관련하여 0 개의 학과와 대학이 검색되었습니다.
< 제조 > 분야와 관련하여 14 개의 학과와 대학이 검색되었습니다.
< 프린팅 > 분야와 관련하여 12 개의 학과와 대학이 검색되었습니다.
< 컴퓨터 > 분야와 관련하여 929 개의 학과와 대학이 검색되었습니다.
< 캐드 > 분야와 관련하여 0 개의 학과와 대학이 검색되었습니다.
< 그래픽스 > 분야와 관련하여 0 개의 학과와 대학이 검색되었습니다.
< 로봇 > 분야와 관련하여 124 개의 학과와 대학이 검색되었습니다.
< 공학 > 분야와 관련하여 5515 개의 학과와 대학이 검색되었습니다.
< 차 > 분야와 관련하여 548 개의 학과와 대학이 검색되었습니다.
        학부_과(전공)명                학교명
46       기계자동차공학과              안동대학교
47       기계자동차공학과              우석대학교
260       자동차공학전공              강원대학교
261      자동차기계공학과              신라대학교
387      미래자동차공학과     

C:\Users\황의지\AppData\Local\Temp\ipykernel_15068\228023947.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(filtered_rows)
C:\Users\황의지\AppData\Local\Temp\ipykernel_15068\228023947.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(filtered_rows)
C:\Users\황의지\AppData\Local\Temp\ipykernel_15068\228023947.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(filtered_rows)
C:\Users\황의지\AppData\Local\Temp\ipykernel_15068\228023947.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(filtered_rows)
C:\Users\황의지\AppData\Local\Temp\ipykernel_15068\

`append()` 메서드 대신에 `pandas.concat()` 함수를 사용하여 DataFrame을 합치는 것을 권장합니다. `pandas.concat()` 함수는 여러 개의 DataFrame을 합쳐 하나의 DataFrame으로 만들어줍니다. 이를 사용하면 `append()` 메서드에 대한 FutureWarning 경고가 발생하지 않을 것입니다.

아래는 `pandas.concat()` 함수를 사용하여 DataFrame을 합치는 방법입니다:

```python
# 빈 DataFrame 생성
new_df = pd.DataFrame()

# 'filtered_morp' 리스트의 각 항목을 순회하면서 DataFrame 'df'에서 해당 학부 또는 전공명을 가진 row들만 추출하여 합치기
for word in filtered_morp:
    filtered_rows = df[df['학부_과(전공)명'].str.contains(word)]
    new_df = pd.concat([new_df, filtered_rows])

# 중복 제거
new_df = new_df.drop_duplicates(subset=['학부_과(전공)명', '학교명'])
```

위 코드를 사용하면 `append()` 메서드를 사용하지 않고 DataFrame을 합치게 되며, 중복된 행들은 `drop_duplicates()` 메서드를 사용하여 삭제됩니다. 이렇게 하면 FutureWarning이 발생하지 않을 것입니다.

In [69]:
#filtered_morp = ['자동', '기계', '감시', '산업', '생체', '의료', '전기', '제어', '디자인', '재료', '제품', '원격', '제조', '프린팅', '컴퓨터', '캐드', '그래픽스', '로봇', '공학', '차']
# 빈 DataFrame 생성
new_df = pd.DataFrame()
df = df.dropna(subset=['학부_과(전공)명'])

# 'filtered_morp' 리스트의 각 항목을 순회하면서 DataFrame 'df'에서 해당 학부 또는 전공명을 가진 row들만 추출하여 합치기
for word in filtered_morp:
    filtered_rows = df[df['학부_과(전공)명'].str.contains(word)]
    new_df = pd.concat([new_df, filtered_rows])
    print('<', word, '> 분야와 관련하여', len(filtered_rows), '개의 학과와 대학이 검색되었습니다.')   # 임의로 작성

# 중복 제거
new_df = new_df.drop_duplicates(subset=['학부_과(전공)명', '학교명'])

# 결과 출력
print(new_df)
print("\n", '해당 분야와 관련하여 총', len(new_df), '개의 학과와 대학이 검색되었습니다.')
#new_df.to_csv('C:/Users/황의지/Desktop/hk+/2', index=False, encoding='utf-8-sig')

< 자동 > 분야와 관련하여 601 개의 학과와 대학이 검색되었습니다.
< 기계 > 분야와 관련하여 925 개의 학과와 대학이 검색되었습니다.
< 감시 > 분야와 관련하여 0 개의 학과와 대학이 검색되었습니다.
< 산업 > 분야와 관련하여 985 개의 학과와 대학이 검색되었습니다.
< 생체 > 분야와 관련하여 11 개의 학과와 대학이 검색되었습니다.
< 의료 > 분야와 관련하여 359 개의 학과와 대학이 검색되었습니다.
< 전기 > 분야와 관련하여 662 개의 학과와 대학이 검색되었습니다.
< 제어 > 분야와 관련하여 147 개의 학과와 대학이 검색되었습니다.
< 디자인 > 분야와 관련하여 1671 개의 학과와 대학이 검색되었습니다.
< 재료 > 분야와 관련하여 87 개의 학과와 대학이 검색되었습니다.
< 제품 > 분야와 관련하여 27 개의 학과와 대학이 검색되었습니다.
< 원격 > 분야와 관련하여 0 개의 학과와 대학이 검색되었습니다.
< 제조 > 분야와 관련하여 14 개의 학과와 대학이 검색되었습니다.
< 프린팅 > 분야와 관련하여 12 개의 학과와 대학이 검색되었습니다.
< 컴퓨터 > 분야와 관련하여 929 개의 학과와 대학이 검색되었습니다.
< 캐드 > 분야와 관련하여 0 개의 학과와 대학이 검색되었습니다.
< 그래픽스 > 분야와 관련하여 0 개의 학과와 대학이 검색되었습니다.
< 로봇 > 분야와 관련하여 124 개의 학과와 대학이 검색되었습니다.
< 공학 > 분야와 관련하여 5515 개의 학과와 대학이 검색되었습니다.
< 차 > 분야와 관련하여 548 개의 학과와 대학이 검색되었습니다.
        학부_과(전공)명                학교명
46       기계자동차공학과              안동대학교
47       기계자동차공학과              우석대학교
260       자동차공학전공              강원대학교
261      자동차기계공학과              신라대학교
387      미래자동차공학과     

` .to_list()`  메서드 -  pandas DataFrame 열을 파이썬 리스트로 변환하는 메서드

` .join()`  메서드 - 는 리스트의 각 항목을 문자열로 결합할 때 사용하는 메서드. 즉, 리스트의 각 항목들 사이에 특정 문자(여기서는 콤마와 공백)를 넣어서 하나의 문자열로 합치는 역할을 합니다. 이때, .join() 메서드의 인자로 들어가는 것은 리스트여야 합니다.

In [71]:
while True:   # 유효헌 대학명 입력할때까지
    answer_uni = input('희망하시는 대학명을 입력하세요(예:중앙대학교):')

    if answer_uni in new_df['학교명'].values:
        print('해당 분야와 관련하여', '<', answer_uni, '>', '에는 총', len(new_df[new_df['학교명'].str.contains(answer_uni)]), '개의 학과가 존재합니다.')
        majors_related_to_uni = new_df['학부_과(전공)명'][new_df['학교명'].str.contains(answer_uni)].to_list()
        print(', '.join(majors_related_to_uni))
        break
    else:
        print('<', answer_uni, '>', '에는 해당 분야 관련 전공이 개설되어 있지 않습니다. 정확한 대학명을 입력해주세요.')   

희망하시는 대학명을 입력하세요(예:중앙대학교):중앙
< 중앙 > 에는 해당 분야 관련 전공이 개설되어 있지 않습니다. 정확한 대학명을 입력해주세요.
희망하시는 대학명을 입력하세요(예:중앙대학교):중앙대학교
해당 분야와 관련하여 < 중앙대학교 > 에는 총 37 개의 학과가 존재합니다.
기계공학부, 디자인학부(산업디자인전공), 산업경제학과, 산업보안학과, 전자전기공학부, 디자인학부(패션디자인전공), 디자인학부(실내환경디자인전공), 디자인학부(시각디자인전공), 디자인학부(공예전공), 디자인학부, 디자인학부(패션전공), 컴퓨터공학부(컴퓨터공학전공), 컴퓨터공학부(소프트웨어전공), 컴퓨터예술학부, 사회기반시스템공학부, 화학신소재공학부, 건축학부(건축공학전공), 사회기반시스템공학부(도시시스템공학전공), 토목공학과, 도시공학과, 사회기반시스템공학부(건설환경플랜트공학전공), 에너지시스템공학부, 생명공학과, 융합공학부, 식품공학부(식품영양전공), 식품공학부(식품공학전공), 식품공학부, 시스템생명공학과, 생명자원공학부(식물시스템과학전공), 생명자원공학부(동물생명공학전공), 생명자원공학부, 생명공학대학, 생명자원공학부(식물생명공학전공), 첨단소재공학과, 예술공학부, 예술공학대학, 차세대반도체학과


In [146]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException, TimeoutException
import pandas as pd
import time

# 요소를 클릭하는 함수 (재시도 포함)
def click_element_with_retries(driver, element, max_retries=2):
    for _ in range(max_retries):
        try:
            element.click()
            return
        except StaleElementReferenceException:
            # StaleElementReferenceException 발생 시 재시도
            continue
    raise StaleElementReferenceException("Stale element reference: Element not found.")

table_data = []

sugang_url = 'https://mportal.cau.ac.kr/std/usk/sUskSif001/search.do'
driver = webdriver.Chrome()
driver.get(sugang_url)

start_time = time.time()
time.sleep(1)

# 2022년 선택
year_option = Select(WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.ID, 'sel_year'))))
year_option.select_by_index(1)
#time.sleep(1)

# 학부 선택
course_option = Select(WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.ID, 'sel_course'))))
course_option.select_by_visible_text('학부')
#time.sleep(1)

for s in range(1, 4):  # 학기
    if s == 1 or s == 3:
        # 학기 선택
        shtm_option = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="sel_shtm"]/option[{s}]')))
        shtm_option.click()
        shtm_value = shtm_option.text.strip()
        time.sleep(1)

        for c in range(2, 4):  # 캠퍼스
            # 캠퍼스 선택
            camp_option = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="sel_camp"]/option[{c}]')))
            camp_option.click()
            time.sleep(1)

            for d in range(3, len(driver.find_elements(By.XPATH, '//*[@id="sel_colg"]/option')) + 1):
                # 단과대 선택
                colg_option = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="sel_colg"]/option[{d}]')))
                colg_option.click()
                time.sleep(1)

                for i in range(1, len(driver.find_elements(By.XPATH, '//*[@id="sel_sust"]/option')) + 1):
                    # 학과(부)/전공/영역 선택
                    try:
                        sust_option = Select(WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.ID, 'sel_sust'))))
                        sust_option.select_by_index(i)
                        sust_value = sust_option.first_selected_option.text.strip()
                    except NoSuchElementException:
                        continue

                    # 검색 버튼을 찾기 위해 //button[contains(text(), "검색")] XPath를 사용하여 WebDriverWait를 통해 해당 요소를 기다림
                    search_button = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "검색")]')))
                    search_button.click()
                    
                    # 로딩 오버레이가 사라질 때까지 대기 (검색 결과가 로드되는 시간을 기다리는 것)
                    #WebDriverWait(driver, 53).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.sp-loading-wrap.none.ng-isolate-scope'))) 
                                                                                                          # 'div.sp-loading-wrap.none.ng-isolate-scope'은 로딩 오버레이를 나타내는 CSS 선택자
                        
                    # 대기 없이 로딩 요소가 사라졌는지 확인하고, 사라졌으면 바로 다음 코드를 실행
                    loading_overlay = driver.find_elements(By.CSS_SELECTOR, 'div.sp-loading-wrap.none.ng-isolate-scope')
                    if not loading_overlay:
                        # 로딩 요소가 사라졌으므로 바로 다음 코드를 실행
                        pass

                    try:
                        # 검색 결과가 로드될 때까지 대기
                        WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.sp-grid-data-row')))
                        time.sleep(1)
                    except TimeoutException:
                        # 데이터가 없는 경우
                        continue

                    # table data
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    data = soup.find_all("div", {'class': 'sp-grid-data-row'})

                    for row in data:
                        row_data = [span.text.strip() for span in row.find_all('span', class_='ng-binding ng-scope')]
                        row_data.insert(0, shtm_value)  # Insert 학기 정보
                        row_data.insert(1, sust_value)  # Insert 학과(부)/전공/영역 정보
                        table_data.append(row_data)

end_time = time.time()  # 데이터 수집 종료 시간 기록

# table data as DataFrame
column_names = ['대학', '학과(부)/전공/영역', '학기', '개설학과', '학년', '과정', '이수구분', '과목번호-분반', '과목명', '학점-시간', '담당교수', '폐강', '강의시간', '유연학기', '비고', '수업유형']
cau_df = pd.DataFrame(data=table_data, columns=column_names)
cau_df = cau_df.drop_duplicates()
#courses_selected_df.drop_duplicates(subset='과목명', keep='first', inplace=True)  -> 원본(w/과목번호)을 살리고 싶어서 과목명 중복제거는 나중에 진행했음 
print(cau_df)
cau_df.to_csv('C:/Users/황의지/Desktop/hk+/cau_df.csv', index=False, encoding='utf-8-sig')

elapsed_time = end_time - start_time
print("데이터 수집에 걸린 시간: ", elapsed_time, "초")

      대학 학과(부)/전공/영역      학기   개설학과  학년  과정  이수구분   과목번호-분반  \
0      1  외식산업경영연계전공  대학(전체)         전체  학사    교양  28703-01   
1      1  외식산업경영연계전공  대학(전체)         전체  학사    교양  28703-01   
2      1  외식산업경영연계전공  대학(전체)         전체  학사    교양  28703-02   
3      1  외식산업경영연계전공  대학(전체)         전체  학사    교양  28703-02   
4      1  외식산업경영연계전공  대학(전체)         전체  학사    교양  28703-03   
...   ..         ...     ...    ...  ..  ..   ...       ...   
18116  2       예술공학부  예술공학대학  예술공학부   3  학사  전공필수  56845-02   
18117  2       예술공학부  예술공학대학  예술공학부   3  학사  전공필수  56845-03   
18118  2       예술공학부  예술공학대학  예술공학부   3  학사    전공  55403-01   
18119  2       예술공학부  예술공학대학  예술공학부   3  학사    전공  55403-02   
18120  2       예술공학부  예술공학대학  예술공학부   4  학사    전공  56084-01   

                     과목명 학점-시간 담당교수 폐강  \
0                  경제와사회   3-3  장선해      
1                  경제와사회   3-3  손주현      
2                  경제와사회   3-3  손주현      
3                  경제와사회   3-3  김지영      
4                  경제와사회   3-3  김

- `timeschedule_df['학과(부)/전공/영역'].str.contains(major_choice)`

: 이 부분은 '학과(부)/전공/영역' 열의 각 행에 대해 `major_choice`가 포함되어 있는지를 확인하는 조건식을 만듭니다. 이 조건식의 결과는 불리언 시리즈로서, `True`와 `False`로 이루어진 시리즈입니다. `True`는 `major_choice`가 해당 행의 '학과(부)/전공/영역' 값에 포함되어 있다는 의미이고, `False`는 포함되어 있지 않다는 의미입니다.

- `timeschedule_df.loc[조건식, ['학기', '학년', '이수구분', '과목명']]`

: 이 부분은 앞에서 만든 조건식을 사용하여 데이터프레임 `timeschedule_df`를 필터링하는 작업을 수행합니다. `loc` 메서드는 행과 열을 라벨로 선택할 수 있도록 도와주는 메서드입니다. 여기서는 조건식을 사용하여 행을 선택하고, 열은 '학기', '학년', '이수구분', '과목명' 열만 선택하도록 합니다. 

In [147]:
#timeschedule_df = pd.read_csv('C:/Users/황의지/Desktop/hk+/cau_df.csv')
#import pandas as pd

courses_df = cau_df  # 시간표 크롤링 한 dataframe 파일

while True:
    answer_uni = input('희망하시는 대학을 입력하세요(예:중앙대학교):')

    if answer_uni in new_df['학교명'].values:
        print('해당 분야와 관련하여', '<', answer_uni, '>', '에는 총', len(new_df[new_df['학교명'].str.contains(answer_uni)]), '개의 학과가 존재합니다.')
        majors_related_to_uni = new_df['학부_과(전공)명'][new_df['학교명'].str.contains(answer_uni)].to_list()
        print(', '.join(majors_related_to_uni))
        break
    else:
        print('<', answer_uni, '>', '에는 해당 분야 관련 전공이 개설되어 있지 않습니다. 정확한 대학명을 입력해주세요.')

while True:
    major_choice = input('희망하시는 전공을 입력하세요.(예:첨단소재공학과/경영학과):')   # 챗봇 사용자 답변(개설과목 정보를 준다는 등의 메시지 추가 작업 필요)
    
    if major_choice in courses_df['학과(부)/전공/영역'].values:
        courses_selected_df = courses_df.loc[courses_df['학과(부)/전공/영역'].str.contains(major_choice), ['학기', '학년', '이수구분', '과목명']]
        courses_selected_df = courses_selected_df.drop_duplicates(subset='과목명', keep='first')
        # == courses_selected_df.drop_duplicates(subset='과목명')
        print(courses_selected_df)
        break

    else:
        print('해당 대학에는 현재', major_choice, '가 존재하지 않습니다. 정확한 전공명을 입력해주세요.(혹은 변동사항이 있을 수 있으니 해당 학교 사이트를 방문해보시길 바랍니다.)')

희망하시는 대학을 입력하세요(예:중앙대학교):중앙대학교
해당 분야와 관련하여 < 중앙대학교 > 에는 총 37 개의 학과가 존재합니다.
기계공학부, 디자인학부(산업디자인전공), 산업경제학과, 산업보안학과, 전자전기공학부, 디자인학부(패션디자인전공), 디자인학부(실내환경디자인전공), 디자인학부(시각디자인전공), 디자인학부(공예전공), 디자인학부, 디자인학부(패션전공), 컴퓨터공학부(컴퓨터공학전공), 컴퓨터공학부(소프트웨어전공), 컴퓨터예술학부, 사회기반시스템공학부, 화학신소재공학부, 건축학부(건축공학전공), 사회기반시스템공학부(도시시스템공학전공), 토목공학과, 도시공학과, 사회기반시스템공학부(건설환경플랜트공학전공), 에너지시스템공학부, 생명공학과, 융합공학부, 식품공학부(식품영양전공), 식품공학부(식품공학전공), 식품공학부, 시스템생명공학과, 생명자원공학부(식물시스템과학전공), 생명자원공학부(동물생명공학전공), 생명자원공학부, 생명공학대학, 생명자원공학부(식물생명공학전공), 첨단소재공학과, 예술공학부, 예술공학대학, 차세대반도체학과
희망하시는 전공을 입력하세요.(예:첨단소재공학과/경영학과):기계공학부
            학기  학년  이수구분                       과목명
2591    대학(전체)  전체    교양                       한국사
2597   다빈치교양대학   1    교양          AI시대문제해결을위한디자인사고
2606      공과대학   1    전공                       CAD
2610      공과대학   1  자유선택                 CAU세미나(1)
2623    대학(전체)   1    교양  COMMUNICATION IN ENGLISH
...        ...  ..   ...                       ...
13051     공과대학   3    전공                   유한요소법개론
13053     공과대학   4    전공  

In [148]:
courses_selected_df

,학기,학년,이수구분,과목명
2591,대학(전체),전체,교양,한국사
2597,다빈치교양대학,1,교양,AI시대문제해결을위한디자인사고
2606,공과대학,1,전공,CAD
2610,공과대학,1,자유선택,CAU세미나(1)
2623,대학(전체),1,교양,COMMUNICATION IN ENGLISH
...,...,...,...,...
13051,공과대학,3,전공,유한요소법개론
13053,공과대학,4,전공,소성가공학
13054,공과대학,4,전공,유체기계및설계
13055,공과대학,4,전공,재료거동
